## Introduction

### UMAP + K-Means Clustering

* `n_neighbors`: This determines the number of neighboring points used in local approximations of manifold structure. Larger values will result in more global structure being preserved at the loss of detailed local structure. In general this parameter should often be in the **range 5 to 50**, with a choice of 10 to 15 being a sensible default.

* `min_dist`: This controls how tightly the embedding is allowed compress points together. **Larger values** ensure embedded points are **more evenly distributed**, while smaller values allow the algorithm to optimise more accurately with regard to local structure. Sensible values are in the range 0.001 to 0.5, with 0.1 being a reasonable default.

* `metric`: This determines the choice of metric used to measure distance in the input space. A wide variety of metrics are already coded, and a user defined function can be passed as long as it has been JITd by numba.

`['euclidean','manhattan','chebyshev','minkowski','canberra','braycurtis','mahalanobis','wminkowski',,'cosine'，'correlation','haversine', 'hamming','jaccard','dice','russelrao', 'kulsinski', 'll_dirichlet', 'hellinger' ]`

euclidean
manhattan
chebyshev
minkowski
canberra
braycurtis
~~mahalanobis~~
wminkowski
~~seuclidean~~
cosine
correlation
haversine
hamming
jaccard
dice
~~russelrao~~
kulsinski
ll_dirichlet
hellinger
rogerstanimoto
sokalmichener
sokalsneath
yule

[UMAP API Guide](https://umap.scikit-tda.org/api.html)

* `n_components`: 2-3

In [1]:
%%time
! pip install hdbscan
# ! conda install -y gdown
# ! gdown --id 1fskNgcHG8kxHIOr6MlZ61OCnDE4DdhgU

     |████████████████████████████████| 5.2 MB 2.0 MB/s            
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - \ | / done
  Preparing metadata (pyproject.toml) ... - \ | / done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=3413272 sha256=b8fd4b13b83b8b6eb269d0e06031735e5ed20425618afad9ca554f238fbe2b15
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
Successfully built hdbscan
CPU times: user 2.31 s, sys: 506 ms, total: 2.82 s
Wall time: 1min 49s


In [2]:
# for everything else
import os
import random
# from random import randint
from functools import partial
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

# for loading/processing the images  
# import tensorflow
# from imageio import imread
# import cv2
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
import pickle
# from keras.layers import Activation, Dense, Dropout

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# clustering and dimension reduction
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import SpectralClustering
import hdbscan

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
# from sklearn.preprocessing import LabelEncoder
# from keras.utils.np_utils import to_categorical
from tqdm import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# dr_methods = ['pca', 't-sne', 'umap']
# clustering_methods = ['agglomerative',
#                       'vanilla kmeans',
#                       'MinibatchKMeans',
#                       'spectral',
#                       'hdbscan ']


## Hyperparameters
* agglomerative: `n_clusters` `affinity`
* k-means: `n_initint`, `max_iter`

In [4]:
config = {
    'method': 'MinibatchKMeans',  #['agglomerative','kmeans','MinibatchKMeans','spectral','hdbscan '],
    'output_features': 512,
#     'dimension_redcution': dr_methods,
#     'clustering': clustering_methods,
    'n_neighbors': range(20,500,20),
    'min_dist': np.arange(0.0, 1.0, 0.05),
    'n_components': 2,
    "umap_metric": ['manhattan','canberra','hellinger','correlation','jaccard','cosine','dice','ll_dirichlet','minkowski'],
    'n_clusters': 4,
    'n_init': 10,
    'batch_size': 512, # 256 * number of cores
    'min_cluster_size': 15,
    'num_evals': 0,
    'max_evals': 1000,
    'random_state': 42
}

In [5]:
'space:', len(config['n_neighbors']) * len(config['min_dist']) * len(config['umap_metric'])

('space:', 4320)

## Load Data

In [6]:
base_dir = '../input/nbiinfframes/FRAMES'
Fold1 = '../input/nbiinfframes/FRAMES/Fold1'
Fold2 = '../input/nbiinfframes/FRAMES/Fold2'
Fold3 = '../input/nbiinfframes/FRAMES/Fold3'

model_path = r'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
pkl_file = r"../input/feature-embedding/files_feature_embeddings_vgg16.pkl"

class_ = ['B', 'I', 'S', 'U']
Folds = [Fold1, Fold2, Fold3]
all_data = []

# Define path to the data directory
for fold in Folds:
    for case in class_:
        path = os.path.join(fold, case)
        if os.path.isdir(path):
            for img in os.listdir(path):
                all_data.append([os.path.join(path, img), case, os.path.basename(fold)])
        
all_df = pd.DataFrame(all_data, columns=['image_path', 'class', 'folder'],index=None)
all_df.dropna(inplace = True)
# all_df.sample(10)

#mapping the image name with the image path
all_df['name'] = all_df['image_path'].map(lambda x: os.path.basename(x))
img_name = all_df['name'] .to_list()
files = all_df['image_path'].to_list()

In [7]:
# all_df['target'] = all_df['class'].astype('category').cat.codes
# all_df.head()

## Methods: Building Model

In [8]:
%%time 

# if os.path.exists(model_path):
#     model = load_model(model_path)
# else:
#     print('model path do not exsit, downloading the model')
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
    
out_features = model.output.shape[1] # to obtain the number of the output feature
# model.summary()
print(out_features)

2022-04-16 12:41:01.993737: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


553476096/553467096 [==============================] - 3s 0us/step
4096
CPU times: user 4.03 s, sys: 3.61 s, total: 7.63 s
Wall time: 5.01 s


## Feature Extraction by CNN

In [9]:
def extract_features(files, model, pkl="", out_feature=out_features):
    
    data = {}
    if os.path.exists(pkl):
        with open(pkl, 'rb') as f:
            data = pickle.load(f)
    else:
        print("File data pkl file doesn't exist..., extracting by model")
        for file in files:
            # load the image as a 224x224 array
            resized_img = load_img(file, target_size=(224,224)) ##reshaped
            # convert from 'PIL.Image.Image' to numpy array
            img_arr = np.array(resized_img) 
            # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
            reshaped_img = img_arr.reshape(1,224,224,3)  ##reshape
            # prepare image for model
            imgx = preprocess_input(reshaped_img)
            ## define the method to extract the feature vector
            embeddings = model.predict(imgx, use_multiprocessing=True)
            data[os.path.basename(file)] = embeddings
            
        pickle.dump(data, open(pkl, "wb"))
 
    # get a list of the filenames
    filenames = np.array(list(data.keys()))
    # get a list of just the features
    feat = np.array(list(data.values()))
    # reshape so that there are 720 samples of 512 vectors
    feat = feat.reshape(-1, out_features)
        
    return filenames, feat

In [10]:
%%time

filenames, feat = extract_features(files, model, pkl_file, out_features)    
print('extracted feature shape: {}'.format(feat.shape))

extracted feature shape: (720, 4096)
CPU times: user 10.6 ms, sys: 14.8 ms, total: 25.4 ms
Wall time: 216 ms


## Different embeddings

In [11]:
# def embed(model, model_type, sentences):
#     if model_type == 'use':
#         embeddings = model(sentences)
#     elif model_type == 'sentence transformer':
#         embeddings = model.encode(sentences, show_progress_bar=True)
    
#     return embeddings

## Different Clusters

In [12]:
def generate_kmclusters(feature_embeddings,
                       n_neighbors,
                       min_dist,
                       metric='euclidean',
                       random_state = config['random_state'],
                       n_components = config['n_components'],
                       method = config['method'],
                       n_clusters = config['n_clusters'],
                       n_init = config['n_init'],batch_size = config['batch_size']):
    
    umap_embeddings = umap.UMAP(n_neighbors = n_neighbors,
                                min_dist = min_dist,
                                n_components = n_components,
                                metric = metric,
                                random_state=random_state).fit_transform(feature_embeddings)
    
    if method == 'kmeans':
        clusters = KMeans(init='k-means++',
                    n_clusters = n_clusters,
                    max_iter=400,
                    n_init=n_init,
                    random_state=random_state).fit(umap_embeddings)
        
    elif method == 'MinibatchKMeans':
        clusters = MiniBatchKMeans(init="k-means++",
                          n_clusters = n_clusters,
                          batch_size = batch_size,
                          max_iter=400,
                          n_init = n_init,
                          max_no_improvement=10,
                          verbose=0,random_state=random_state).fit(umap_embeddings)

    return clusters

In [13]:
def generate_scclusters( feature_embeddings,
                       n_neighbors,
                       min_dist,
                       metric='euclidean',
                       random_state = config['random_state'],
                       n_components = config['n_components'],
                       n_clusters = config['n_clusters'],
                       min_cluster_size = config['min_cluster_size']):
        
    umap_embeddings = umap.UMAP(n_neighbors = n_neighbors,
                                min_dist = min_dist,
                                n_components = n_components,
                                metric = metric,
                                random_state = random_state).fit_transform(feature_embeddings)
    
    sc_umap_cluster = SpectralClustering(n_clusters = n_clusters,
                                       n_neighbors = config['n_neighbors'],
                                       affinity='precomputed',
                                       n_init=100, 
                                       assign_labels='discretize',
                                      random_state=random_state).fit(umap_embeddings)

    clusters = sc_umap_cluster
    return clusters

In [14]:
def generate_hdclusters( feature_embeddings,
                       n_neighbors,
                       min_dist,
                       metric='euclidean',
                       random_state = config['random_state'],
                       n_components = config['n_components'],
                       n_clusters = config['n_clusters'],
                       min_cluster_size = config['min_cluster_size']):

    print('the using metric {}'.format(metric))
    
    umap_embeddings = umap.UMAP(n_neighbors = n_neighbors,
                                min_dist = min_dist,
                                n_components = n_components,
                                metric = metric,
                                random_state=random_state).fit_transform(feature_embeddings)

    hd = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size ,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)
        
    clusters = hd
    return clusters

### Score

In [15]:
def metrics_score_(clusters, filenames, image_label_df):
    
    """
    Returns  objects after first performing dimensionality reduction using UMAP
    
    Arguments:
        clusters: clustering object
        filenames: image name list
        image_label_df: dataframe object, the image name and the groundtruth label
        
    Returns:
        cluster_labels: K-Means object of clusters
    """
    mapped_label = {}
    ### prepare the labels from clusters
    clustered_labels = clusters.labels_ #clustered labels
    clustered_groups = {}
    for file, cluster in zip(filenames, clustered_labels):
        if cluster not in clustered_groups.keys():
            clustered_groups[cluster] = []
            clustered_groups[cluster].append(file)
        else:
            clustered_groups[cluster].append(file)
    
    
    ### prepare the ground truth labels, using group by 
    image_groups = image_label_df.groupby('class')['name'] ## only get the image name for matching the predicted one
    ground_labels = [*image_groups.groups.keys()] 
    
    ## mapping each group in datafame with the max intersection clustered groups
    for label in ground_labels:
        group_label = {} # allocate to the max intersection number group
        lst = image_groups.get_group(label).tolist()
        
        for x in range(len(clustered_groups)):
            num = len(set(clustered_groups[x]) & set(lst))
#             print('clustered group: {} intersection with label {}, instersection number {}'.format(x, label, num))
            group_label[x]=num
            
        max_key = max(group_label, key=group_label.get)
#         print('the {} label is the group {}'.format(label, max_key))
        mapped_label[max_key] = label #reverse the label
        
        ### connect the mapped labels to the image name
        np_groups = np.array(list(clustered_groups.items()))
        imgs = []
        clustered_img_labels = []
        for i in range(np_groups.shape[0]):
            imgs.extend(np_groups[i,:][1])
            clustered_img_labels.extend([np_groups[i,:][0]]*len(np_groups[i,:][1]))
        
        ## merge the mapped df with the dataframe
        clustered_df = pd.DataFrame({'name': imgs, 'predict_target': clustered_img_labels})
        clustered_df['predict_class'] = clustered_df['predict_target'].map(mapped_label)
        eval_df = pd.merge(image_label_df, clustered_df, on='name', how='left')
        
        y_test = eval_df['class'].tolist()
        y_pred = eval_df['predict_class'].tolist()
        
        report = classification_report(y_test, y_pred)

        precision_score_ = precision_recall_fscore_support(y_test, y_pred)[0]
        recall_score_ = precision_recall_fscore_support(y_test, y_pred)[1]
        f1_score_ = precision_recall_fscore_support(y_test, y_pred)[2]

        avg_precision = np.average(precision_score_)
        avg_recall = np.average(recall_score_)
        avg_f1 = np.average(f1_score_)
        
    return avg_precision, avg_recall, avg_f1

In [16]:
# %time clusters = generate_kmclusters(feat, min_dist=0, metric='euclidean') #default: euclidean
# %time cost = score_(clusters, filenames, all_df)

In [17]:
def objective(params, embeddings, filenames, df):
    """
    Objective function for hyperopt to minimize, which incorporates constraints
    on the number of clusters we want to identify
    """
    
    clusters = generate_kmclusters(embeddings,
                                   n_neighbors = params['n_neighbors'], 
                                   min_dist = params['min_dist'], 
                                   metric = params['umap_metric'])
#     print(params['umap_metric'])
    
    avg_precision, avg_recall, avg_f1 = metrics_score_(clusters, filenames, df)
    cost = 1 - avg_recall 
    
    if (np.abs(avg_precision - avg_recall) > 0.025):
        penalty = 0.1 * np.abs(avg_precision - avg_recall)
    else:
        penalty = 0
    loss = cost + penalty # penalty
    return {'loss': loss, 'avg_precision': avg_precision,'avg_recall': avg_recall, 'avg_f1':avg_f1, 'status': STATUS_OK}

In [18]:
def bayesian_search(features, filenames, df, space, max_evals=100):

    """
    Perform bayseian search on hyperopt hyperparameter space to minimize objective function
    """
    
    trials = Trials()
    
    fmin_objective = partial(objective, embeddings=features, filenames=filenames, df=df)
    
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"the average recall: {trials.best_trial['result']['avg_recall']}")

    best_clusters = generate_kmclusters(features, 
                                      n_neighbors = best_params['n_neighbors'], 
                                      min_dist = best_params['min_dist'], 
                                      metric = best_params['umap_metric'])
    

    return best_params, best_clusters, trials

In [19]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', config['n_neighbors']),
    "min_dist": hp.choice('min_dist', config['min_dist']),
    "umap_metric": hp.choice('umap_metric', config['umap_metric']),
}

max_evals=config['max_evals']

In [20]:
print('test the time-cost for the method {}'.format(config['method']))
%time best_params_use, best_clusters_use, trials_use = bayesian_search(feat, filenames, all_df, hspace, max_evals)

test the time-cost for the method MinibatchKMeans
100%|██████████| 1000/1000 [3:35:38<00:00, 12.94s/trial, best loss: 0.07222222222222219]
best:
{'min_dist': 0.30000000000000004, 'n_neighbors': 360, 'umap_metric': 'canberra'}
the average recall: 0.9277777777777778
CPU times: user 3h 36min 9s, sys: 1min 48s, total: 3h 37min 58s
Wall time: 3h 35min 52s


In [21]:
trials_use.best_trial

{'state': 2,
 'tid': 57,
 'spec': None,
 'result': {'loss': 0.07222222222222219,
  'avg_precision': 0.9310420307600001,
  'avg_recall': 0.9277777777777778,
  'avg_f1': 0.9262374103130735,
  'status': 'ok'},
 'misc': {'tid': 57,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'min_dist': [57], 'n_neighbors': [57], 'umap_metric': [57]},
  'vals': {'min_dist': [6], 'n_neighbors': [17], 'umap_metric': [1]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2022, 4, 16, 12, 51, 43, 597000),
 'refresh_time': datetime.datetime(2022, 4, 16, 12, 51, 56, 846000)}

In [22]:
 umap.__version__

'0.5.2'

## Results

In [23]:
# random_use

In [24]:
best_params_use

{'min_dist': 0.30000000000000004,
 'n_neighbors': 360,
 'umap_metric': 'canberra'}

## Under Evaluation

In [25]:
def random_search(features, space, num_evals, filenames, all_df):
    """
    Randomly search parameter space of clustering pipeline

    Arguments:
        features: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               and 'metrics' and values with
               corresponding lists or ranges of parameters to search
        num_evals: int, number of random parameter combinations to try

    Returns:
        df_result: pandas dataframe containing info on each evaluation
                   performed, including run_id, parameters used, label
                   count, and cost
    """
    
    results = []
    
    for i in trange(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        min_dist = random.choice(space['min_dist'])
        metric = random.choice(space['metric'])
        
        clusters = generate_kmclusters(features,
                                     n_neighbors=n_neighbors, 
                                     min_dist=min_dist,
                                     metric=metric)
    
        avg_precision, avg_recall, avg_f1 = metrics_score_(clusters, filenames, all_df)
        print('round: {}, current setting: n_neighbors: {}, min_dist: {}, metric: {}'.format(i,n_neighbors,
                                                                                             min_dist,
                                                                                             metric))
        print('### precsion: {}, recall: {}, f1-measure: {} ###'.format(np.round(avg_precision,2),
                                                                                  np.round(avg_recall,2),
                                                                                 np.round(avg_f1,2)))
        results.append([i, n_neighbors, min_dist, metric, avg_precision, avg_recall, avg_f1])
        
        
    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'min_dist', 
                                               'metric', 'avg_precision', 'avg_recall', 'avg_f1'])
    
    return result_df.sort_values(by='avg_recall', ascending=False)

In [26]:
space = {
        "n_neighbors": config['n_neighbors'],
        "min_dist": config['min_dist'],
        "metric": config['umap_metric'] 
    }

# n_neighbors, min_dist, metric
# %time random_use = random_search(feat, space, config['num_evals'] , filenames, all_df)
# random_use.to_csv('umap_hyperparameters_random_search_{}.csv'.format(num_evals))